In [ ]:
import pandas as pd

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics
import numpy as np

In [ ]:
from multiprocessing import Pool

In [ ]:
def run_calculation(thread_option_list):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            p.map(proc.calc,
                  thread_option_list)

In [ ]:
flash_names = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*данные с флеш-карты\\*')

In [ ]:
for flash_name in flash_names:
    print(flash_name)
    this_flash_data_df = workflow_cs_data.read_control_station_data(flash_name)
    this_path_edited_data = flash_name.replace('.xlsx', '_init_edit.csv')
    print(this_path_edited_data)
    this_flash_data_df.to_csv(this_path_edited_data)